In [355]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk.stem import WordNetLemmatizer

import unittest
import numpy as np
from tinygrad.tensor import Tensor, Device
from tinygrad.nn import optim, BatchNorm2d
from extra.training import train, evaluate
from datasets import fetch_mnist

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

import time
from tinygrad.tensor import Tensor
from tinygrad.nn import Linear
from tinygrad.nn.optim import Adam
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rando\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rando\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rando\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rando\AppData\Roaming\nltk_data...


True

In [386]:
train_data.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [ ]:


train_data = pd.read_csv('data_tw/train.csv').set_index("id")
train_data.head()

def preprocess_text(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    return ' '.join(words)

def embed(text):
    emb = np.zeros(18613)
    for i in text:
        try:
            emb[voc.index(i)]=1
        except:
            pass 
    return emb


test = pd.read_csv('data_tw/test.csv').set_index("id")
train_data['text'] = train_data['text'].apply(preprocess_text)
test['text'] = test['text'].apply(preprocess_text)


all_text = ' '.join(train_data['text'])
all_text += ' '.join(test['text'])

voc = list(set(all_text.split()))

train_data['emb'] = train_data['text'].apply(embed)

X_train = np.stack(train_data['emb'].apply(lambda x: [int(i) for i in x]).values).astype(np.float32)
Y_train = train_data['target'].values.astype(np.uint8)

class TwitterNet:
    def __init__(self):
        self.l1 = Tensor.scaled_uniform(18613, 512)  # First hidden layer has 512 nodes
        self.l2 = Tensor.scaled_uniform(512, 256)
        self.l3 = Tensor.scaled_uniform(256,100)    # Second hidden layer has 256 nodes
        self.l4 = Tensor.scaled_uniform(100, 2)      # Output layer has 2 nodes

    def parameters(self):
        return optim.get_parameters(self)

    def forward(self, x):
        x = x.dot(self.l1).leakyrelu()        # Input to first hidden layer
        x = x.dot(self.l2).leakyrelu()
        x = x.dot(self.l3).leakyrelu()        # First hidden layer to second hidden layer
        return x.dot(self.l4).log_softmax()  # Second hidden layer to output

model = TwitterNet()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train(model, X_train, Y_train, optimizer, BS=100, steps=1000)                                           



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rando\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rando\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [362]:
# Preprocessing
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    text = text.lower()
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    return ' '.join(words)

# Feature Engineering
def get_word_count(text):
    return len(text.split())

train_data = pd.read_csv('data_tw/train.csv').set_index("id")
train_data['text'] = train_data['text'].apply(preprocess_text)
train_data['word_count'] = train_data['text'].apply(get_word_count)
train_data['has_keyword'] = ~train_data['keyword'].isnull()
train_data['has_location'] = ~train_data['location'].isnull()

test = pd.read_csv('data_tw/test.csv').set_index("id")
test['text'] = test['text'].apply(preprocess_text)
test['word_count'] = test['text'].apply(get_word_count)
test['has_keyword'] = ~test['keyword'].isnull()
test['has_location'] = ~test['location'].isnull()

all_text = ' '.join(train_data['text'])
all_text += ' '.join(test['text'])

voc = list(set(all_text.split()))

def embed(text):
    emb = np.zeros(18613)
    for i in text:
        try:
            emb[voc.index(i)]=1
        except:
            pass 
    return emb

train_data['emb'] = train_data['text'].apply(embed)

# Update feature vector to include additional features
def get_feature_vector(row):
    emb = list(embed(row['text']))
    emb.append(row['word_count'])
    emb.append(int(row['has_keyword']))
    emb.append(int(row['has_location']))
    return emb

X_train = np.stack(train_data.apply(get_feature_vector, axis=1).values).astype(np.float32)
Y_train = train_data['target'].values.astype(np.uint8)


In [363]:
class TwitterNet:
    def __init__(self):
        self.l1 = Tensor.scaled_uniform(18613 + 3, 512)  # Adjust input size to match feature vector
        self.l2 = Tensor.scaled_uniform(512, 256)
        self.l3 = Tensor.scaled_uniform(256,100) 
        self.l4 = Tensor.scaled_uniform(100, 2)  

    def parameters(self):
        return optim.get_parameters(self)

    def forward(self, x):
        x = x.dot(self.l1).relu()       
        x = x.dot(self.l2).relu()
        x = x.dot(self.l3).relu()        
        return x.dot(self.l4)

model = TwitterNet()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train(model, X_train, Y_train, optimizer, BS=100, steps=1000) 

loss -13.74 accuracy 0.63: 100%|██████████| 1000/1000 [01:29<00:00, 11.19it/s]


In [ ]:
train_df = pd.read_csv('data_tw/train.csv')
test_df = pd.read_csv('data_tw/test.csv')
train_df['text'] = 'TEXT: ' + train_df['text'].fillna(' ') + ' LOCATION: ' + train_df['location'].fillna(' ') + ' KEYWORD: ' + train_df['keyword'].fillna(' ')
test_df['text'] = 'TEXT: ' + test_df['text'].fillna(' ') + ' LOCATION: ' + test_df['location'].fillna(' ') + ' KEYWORD: ' + test_df['keyword'].fillna(' ')
train_df.drop(['location', 'keyword', 'id'], axis=1, inplace=True)
train_df.rename(columns={'target': 'label'}, inplace=True)
train_df['is_valid'] = False

test_df.drop(['location', 'keyword', 'id'], axis=1, inplace=True)
test_df.rename(columns={'target': 'label'}, inplace=True)

train_df.head()
num_rows_to_change = int(len(train_df) * 0.2)
random_indices = np.random.choice(train_df.index, size=num_rows_to_change, replace=False)
train_df.loc[random_indices, 'is_valid'] = True

from fastai.text.all import *

twitter_cl = DataBlock(blocks=(TextBlock.from_df('text', seq_len=72), CategoryBlock),
                      get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

dls = twitter_cl.dataloaders(train_df, bs=64)
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)
test_df['label'] = 0
t = test_df

# Assume df is your DataFrame and 'text' is the column containing text
test_dl = learn.dls.test_dl(t['text'])  # Create a test DataLoader
preds, _ = learn.get_preds(dl=test_dl)  # Make predictions

predicted_labels = preds.argmax(dim=1)
t['label'] = predicted_labels

t['label'] = t['text'].apply(learn.predict)

learn.predict('TEXT: EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTENERS XrWn LOCATION:   KEYWORD')
sub = pd.read_csv('data_tw/sample_submission.csv').set_index('id')
sub['target'] = t.label
out = t.drop(['text'], axis=1, inplace=False)
out = out.rename(columns={'label': 'target'})
out = out.reset_index()

# Rename the new column (which is named 'index' by default) to 'id'
out = out.rename(columns={'index': 'id'})
out
out.to_csv('data_tw/out_y.csv', index=False)
pd.read_csv('data_tw/out_y.csv').head()

In [580]:
len(out)

3263

In [581]:
import numpy as np
import pandas as pd
from fastai.text.all import *

# Load data
train_df = pd.read_csv('data_tw/train.csv')
test_df = pd.read_csv('data_tw/test.csv')

# Preprocess text
for df in [train_df, test_df]:
    df['text'] = 'TEXT: ' + df['text'].fillna(' ') + \
                 ' LOCATION: ' + df['location'].fillna(' ') + \
                 ' KEYWORD: ' + df['keyword'].fillna(' ')
    df.drop(['location', 'keyword'], axis=1, inplace=True)
    df.rename(columns={'target': 'label'}, inplace=True)


# Set validation split
num_rows_to_change = int(len(train_df) * 0.2)
random_indices = np.random.choice(train_df.index, size=num_rows_to_change, replace=False)
train_df['is_valid'] = False
train_df.loc[random_indices, 'is_valid'] = True

# Define DataBlock and DataLoader
twitter_cl = DataBlock(blocks=(TextBlock.from_df('text', seq_len=72), CategoryBlock),
                      get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())
dls = twitter_cl.dataloaders(train_df, bs=64)

# Train model
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(2, 1e-2)

# Make predictions on test set
test_dl = learn.dls.test_dl(test_df['text'])  # Create a test DataLoader
preds, _ = learn.get_preds(dl=test_dl)  # Make predictions
predicted_labels = preds.argmax(dim=1)
test_df['label'] = predicted_labels

# Prepare submission
sub = pd.read_csv('data_tw/sample_submission.csv').set_index('id')
sub['target'] = test_df.label
out = test_df.drop(['text'], axis=1, inplace=False)
out = out.rename(columns={'label': 'target'})

# Save to csv
out.to_csv('data_tw/out_y.csv', index=False)
pd.read_csv('data_tw/out_y.csv').head()

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


epoch,train_loss,valid_loss,accuracy,time
0,0.579023,0.497932,0.767411,00:12


epoch,train_loss,valid_loss,accuracy,time
0,0.522218,0.472355,0.773982,00:30
1,0.463258,0.447319,0.793693,00:31


,id,target
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
